# Module 11. 特徵擷取(SIFT)

## 11-1: 擷取原理

* 要辨識某物體的條件就是先掌握其特徵！由於我們要辨識的是某個物件而非整張相片，因此需要提取所謂稱為「Local features」的特徵，作法是先在影像中`選取重要的特徵點`，接著以其為 base 取得周圍的特徵（即local features），這些來自不同相片的 local features 會透過稍後會說明的 Feature matching 功能來比對是否有相同的物件。

* 特徵點 → 局部特徵 → Feature matching 這些特性可用 `edges、corners、blobs (斑點)` 等組合來描述 Keypoint detection、Feature extraction 以及 Feature matching

> * Keypoint detection ：在圖片中取得感興趣的關鍵點（可能為 edges、corners 或 blobs）。
> * Feature extraction ：針對各關鍵點提取該區域的 features（我們稱為 local features）。
> * 關鍵點篩選並進行 Feature matching。

> https://gilscvblog.com/2013/08/18/a-short-introduction-to-descriptors/

## 11-2: Keypoint Descriptor
### FAST – （Features from Accelerated Segment Test）演算法 FastFeatureDetector
> 原理簡單, 執行速度相當快, 主要用於偵測 corners，亦可偵測 blob。適用於要求速度的 real-time analysis, 適用於`速度慢或較低階的執行環境`。使用度極高，尤其在需要即時的 realtime 環境。

> * FAST 方法認為，一個以 p 為中心、半徑為 r 的圓形，若它位於一個所謂的 corner上，那麼該圓的圓周上必有連續 n 個點，其強度值`大於或小於中心點 p `加上一個指定的 threshold 門檻的強度值。如果是的話，該中心點 p 便被認為是 keypoint。

> * 考慮是否應該將中心像素 p 視為關鍵點。中心像素 p 具有灰度強度值 p ＝ 32。為了使該像素成為關鍵點，必須在圓的邊界上具有 n = 12 / 16 (一般 `3/4`) 個連續像素，`這些像素要不就比 p + t 亮，要不就比 p – t 暗`。在此示例中，假設 t = 16。

><img src="./image/KpFast01.png"  style='width:90%'></img>

><img src="./image/KpFast03.png"  style='width:50%'></img>

> FAST演算法提取角點的步驟：
> * 在圖像中選擇圖元 p，假設其灰度值為：Ip
> * 設置一個閾值 T，例如：Ip 的20%
> * 選擇 p 周圍半徑為 3 的圓上的 16 個圖元，作為比較圖元
> * 假設選取的圓上有`連續`的 N 個圖元`大於Ip+T或者小於 Ip−T`，那麼可以認為圖元 p 就是一個特徵點。（N 通常取12，即為 FAST-12；常用的還有 FAST-9, FAST-11）。

> 為了獲得更快的效果，也採用了而外的加速辦法。首先對候選點的周圍每個90 度的點：1，9，5，13 進行測試（先測試1 和19, 如果它們符合閾值要求再測試5 和13）。如果p 是角點，那麼這四個點中至少有3 個要符合閾值要求。如果不是的話肯定不是角點，就放棄吧。

> 缺點 :
> * 檢測到的特徵點過多並且會出現 `紮堆` 的現象。這可以在第一遍檢測完成後，使用非最大值抑制（Non-maximal suppression），在`一定區域內僅保留回應極大值的角點`，避免角點集中的情況。
> * FAST 提取到的角點`沒有方向和尺度`資訊
> * SIFT 和 SURF 演算法都包含有各自的特徵點描述子的計算方法，而 FAST 不包含特徵點描述子的計算，僅僅只有特徵點的提取方法，這就需要一個特徵點描述方法來描述 FAST 提取到的特徵點，以方便特徵點的匹配

https://docs.opencv.org/3.4/df/d74/classcv_1_1FastFeatureDetector.html

## FAST
> kp：關鍵點信息，包括位置，尺度，方向信息

In [ ]:
import numpy as np
import cv2
  
# load the image and convert it to grayscale
image = cv2.imread('./image/lenaColor.png')
# image = cv2.imread('./image/fruits.png')

orig = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
fast = cv2.FastFeatureDetector_create(threshold=None, nonmaxSuppression=True)  # Fast feature detector
kps = fast.detect(gray, None)

print(f'keypoint counts\t\t: {len(kps)}\n'
      f'type\t\t\t: {type(kps)}\n'
      f'Threshold\t\t: {fast.getThreshold()}\n'
      f'nonmaxSuppression\t: {fast.getNonmaxSuppression()}\n'
      # f'neighborhood\t\t: {fast.getType()}\n\n'
      f'kps[:10] :\n{kps[:10]}')
# Print all default params

image = cv2.drawKeypoints(gray, kps, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)   # image : image output

cv2.imshow('Images', np.hstack([orig, image]))
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### SIFT : Scale Invariant Feature Transform，尺度不變特徵變換。
> SIFT特徵對`旋轉、尺度縮放、亮度變化等保持不變性`，是一種非常穩定的局部特徵。

> Fast Keypoint 在影像旋轉的情況下也可以檢測到，但是如果減小(或者增加)影像的大小，可能會丟失影像的某些部分，甚至導致檢測到的角點發生改變。這樣的損失現像`需要一種與影像比例無關`的角點檢測方法來解決。尺度不變特徵變換 (Scale-Invariant Feature Transform, SIFT) 可以解決這個問題。

> SIFT 演算法利用 DoG (差分高斯)來提取關鍵點(或者說成特徵點)，DoG 的思維是用不同的尺度空間因子 (高斯正態分佈的標準差σ) 對影像進行平滑，然後比較平滑後圖像的區別，差別大的畫素就是特徵明顯的點，即可能是特徵點。對得到的所有特徵點，我們剔除一些不好的，SIFT運算元會把剩下的每個特徵點用一個128維的特徵向量進行描述

> * 原理較為複雜。
> * 主要針對 blob 偵測，不過亦可偵測 corners。
> * 可適應物件的 scale (大小變化)及 angle（旋轉角度）等情況。
> * 在DoG模型中，使用了`不同尺寸影像並套用高斯模糊`，`比較不同模糊比例之間的變化`，來決定是否為 keypoint。
> * 由於計算量大，DoG 的執行速度較慢，不適用於 realtime 的環境。
> * SIFT已廣泛的應用於電腦視覺領域，並成為評定新 keypoint detecter 的效率指標。

> 速度上 `ORB > SURF > SIFT，SURF的魯鈍性(抗干擾能力）更好一些`。

><img src="./image/sift_dog.jpg"  style='width:90%'></img>
><img src="./image/SIFT.png"  style='width:90%'></img>
https://aishack.in/tutorials/sift-scale-invariant-feature-transform-scale-space/

> ### SIFT 演算法: 
> 1. 準備輸入圖像的不同比例的副本。為每個比例創建八度比例圖像。
> 2. 對於每個八度比例，高斯模糊濾鏡會應用到所有強度增加的圖像。
> 3. 對八度內的所有兩個後續圖像計算高斯差（DoG）。
> 4. 掃描中間DoG相對於 26 鄰域的極值。通過在極值上應用高通濾波器來檢索特徵。
> 5. 確定特徵的`漸變，方向和位置`。

## SIFT

In [ ]:
import numpy as np
import cv2
gray = cv2.imread('./image/lenaColor.png', 0)
sift=cv2.xfeatures2d.SIFT_create()    # create object
# sift = cv2.SIFT_create()
kp = sift.detect(gray, None)
print(f'kp[:5]\t:\n{kp[:5]}\n\n'
      f'len(kp)\t\t: {len(kp)}\n'
      f'kp[0].pt\t: {kp[0].pt}\n'          # 座標
      f'kp[0].size\t: {kp[0].size}')       # 強度

img = cv2.drawKeypoints(gray, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)   # output to 'img'
# img = cv2.drawKeypoints(gray, kp, None)   # out to 'img'
cv2.imshow('sift_kp', img)

cv2.imwrite('./image/lena_sift_kp.jpg', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### sift / save keypoint

In [ ]:
import numpy as np
import cv2

gray = cv2.imread('./image/mybaby.jpg', 0)
# gray = cv2.imread('./image/lenaColor.png', 0)

sift = cv2.xfeatures2d.SIFT_create()
# kp = sift.detect(gray,None)

# directly find keypoints and descriptors in a single step
kp, des = sift.detectAndCompute(gray, None)   # kp指關鍵點, des指關鍵點的特徵描述
print(f'kps count : {len(kp)}\ndes[0] :\n{des[0:2]}')
img = cv2.drawKeypoints(gray, kp, None)

np.save('./image/mybaby_sift_kp', des)
cv2.imshow('SIFT', img)
# cv2.waitKey(0)

cv2.imwrite('./image/mybaby_sift.jpg', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### SURF : This algorithm `is patented`
Speeded Up Robust Features。加速版的SIFT。
SURF的流程和 SIFT 比較類似，這些改進體現在以下幾個方面：<br>
 * 特徵點檢測是基於 Hessian 矩陣，依據 Hessian 矩陣行列式的極值來定位特徵點的位置。
 * 並且將 Hession 特徵計算與高斯平滑結合在一起，兩個操作通過近似處理得到一個核範本。
 * 在構建尺度空間時，使用`box filter`與源圖像卷積，而不是使用DoG運算元。
 * SURF 使用一階 Haar 小波在 x、y 兩個方向的回應作為構建特徵向量的分佈資訊。

SURF 演算法比 SIFT 快好幾倍，它吸收了 SIFT 演算法的思想。SURF 採用 Hessian 演算法檢測關鍵點。SURF 需要提供閾值，特徵隨著閾值的增加而減少。

In [ ]:
import numpy as np
import cv2
gray = cv2.imread('./image/lenaColor.png', 0)

surf = cv2.xfeatures2d.SURF_create()
# Find keypoints and descriptors directly
kp, des = surf.detectAndCompute(gray, None)
len(kp)

### Brief
過程如下：

 * 為減少雜訊干擾，先對圖像進行高斯濾波（方差為2，高斯窗口為 9x9）<br>
 * 以特徵點為中心，取 SxS 的鄰域大視窗。在大視窗中隨機選取一對（兩個）5x5 的子視窗，比較子視窗內的圖元和（可用積分圖像完成），進行二進位賦值.（一般 S=31）其中，p(x)，p(y)分別隨機點 x=(u1, v1), y=(u2, v2) 所在 5x5 子視窗的圖元和.<br>
 * 在大視窗中隨機選取N對子視窗，重複步驟2 的二進位賦值，形成一個二進位編碼，這個編碼就是對特徵點的描述，即特徵描述子.（一般 N=256）<br>
非常重要的一點是：BRIEF 是一種特徵描述符，它不提供查找特徵的方法。所以我們不得不使用其他特徵檢測器，比如 SIFT 和 SURF 等。原始文獻推薦使用 CenSurE 特徵檢測器，這種演算法很快。而且 BRIEF 演算法對 CenSurE 關鍵點的描述效果要比 SURF 關鍵點的描述更好。

簡單來說 BRIEF 是一種對特徵點描述符計算和匹配的快速方法。這種演算法可以實現很高的識別率，除非出現平面內的大旋轉。

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = cv2.imread('./image/lenaColor.png',0)
star = cv2.xfeatures2d.StarDetector_create()
kp = star.detect(img,None)

brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()  # 初始化BRIEF特徵提取器
kp, des = brief.compute(img, kp)                          # 計算特徵描述
img = cv2.drawKeypoints(img, kp, None, color=-1)

cv2.imshow('Brief', img)

print(f'size\t\t: {brief.descriptorSize()}\n\n'
      f'des[0]\t\t:\n{des[0]}\n\n'
      f'des.shape\t:\n{des}\n\n'
      f'len\t\t: {len(kp)}\n')
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Star

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

img = cv2.imread('./image/lenaColor.png',0)

star = cv2.xfeatures2d.StarDetector_create()               # 初始化STAR檢測器
kp = star.detect(img,None)                                # 使用STAR尋找特徵點
img = cv2.drawKeypoints(img, kp, None, color=-1)
print(len(kp))
cv2.imshow('Star', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### ORB : ORiented Brief / Oriented FAST and Rotated BRIEF
> ORB亦是針對 FAST 的強化，但除了scale space invariance，亦加入了旋轉不變性(rotation invariance)。

> ORB原理有三大步驟：
> * Pyramid 圖像尺寸並進行各尺寸的FAST計算。
> * 使用 Harris keypoint detector的方法計算每個 keypoint分數（是否近似corner？），並進行排序，最多僅取500個keypoints，其餘則丟棄。
> * 於此第三步中加入旋轉不變性，使用「intensity centroid」計算每個 keypoint 的 rotation。ORB 與 BRISK 相同，繼承了 FAST 運算快速的特性，可適用於realtime分析。

ORB 是用來取代 SIFT 和 SURF 的，與兩者相比，ORB有更快的速度。ORB 用FAST來檢測關鍵點，用 BRIEF 來進行關鍵點特徵描述。

In [ ]:
import cv2
import numpy as np
# image = cv2.imread('./image/blox.jpg')
image = cv2.imread('./image/lenaColor.png')

# image = cv2.imread('./image/lenaColor.png', cv2.COLOR_BGR2GRAY)

sift_feature = cv2.xfeatures2d.SIFT_create()
sift_kp = sift_feature.detect(image)
sift_out = cv2.drawKeypoints(image, sift_kp, None)

orb_feature = cv2.ORB_create()
orb_kp  = orb_feature.detect(image)
orb_out  = cv2.drawKeypoints(image, orb_kp, None)

font = 2;    lt = 16
loc = (10, 40); color = (255, 255, 255)
cv2.putText(image, 'original', loc, font, 1, color, 2, lt)
cv2.putText(sift_out, 'sift', loc, font, 1, color, 2, lt)
cv2.putText(orb_out, 'orb', loc, font, 1, color, 2, lt)

image = cv2.hconcat([image, sift_out, orb_out])

cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Harris
> 速度相當快，但仍較 FAST 慢，不過偵測 corner 比起 FAST 準確一些。廣泛應用於偵測 edges 及 corners, Harris 可用於識別角點。此函數可以很好的檢測角點，這些角點在圖像旋轉的情況下也能被檢測到。但是如果減少或者增加圖像的尺寸，可能會丟失圖像的某些部分，也有可能增加圖像的角點。

In [ ]:
import numpy as np
import cv2

img = cv2.imread('./image/lenaColor.png', 1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

dst = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.04)  
print(len(dst))
# blockSize:檢測的臨點數, ksize:sobel邊緣檢測的核, k:目標函式的一個引數（一般取值較小）

#result is dilated for marking the corners, not important
# dst = cv2.dilate(dst, None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]
cv2.imshow('dst', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

## 11-3: Keypoint Matching

### match knn
> BFmatcher (暴力匹配)：計算匹配圖層的一個特徵描述子與待匹配圖層的所有特徵描述子的距離返回最近距離。<br>
> FlannBasedMatcher：是目前最快的特徵匹配演算法（最近鄰搜尋）

那麼這個這個DMatch資料結構究竟是什麼呢？<br>
它包含三個非常重要的資料分別是 queryIdx，trainIdx，distance
* queryIdx：測試圖像的特徵點描述符的下標（第幾個特徵點描述符），同時也是描述符對應特徵點的下標。
* trainIdx：樣本圖像的特徵點描述符下標,同時也是描述符對應特徵點的下標。
* distance：代表這怡翠匹配的特徵點描述符的`歐式距離`，數值越小也就說明倆個特徵點越相近。

In [ ]:
import cv2
# ======== example 0 =================
# img1 = cv2.imread('./image/box.png')
# img2 = cv2.resize(cv2.imread('./image/box_in_scene.png'), None, fx=1.4, fy=1.4)

# ======== example 1 =================
# img1 = cv2.imread('./image/testpic.jpg')
# img2 = cv2.imread('./image/testpic2.jpg')

# ======== example 2 =================
# img1 = cv2.imread('./image/aiotbooks.jpg')
# img2 = cv2.imread('./image/aiotimage.jpg')

# ======== example 3 =================
img1 = cv2.imread('./image/mario.jpg')
img2 = cv2.imread('./image/marioCoin.jpg')

feature = cv2.xfeatures2d.SIFT_create()
# feature = cv2.xfeatures2d.SURF_create()   # This algorithm is patented and is excluded in this configuration

kp1, des1 = feature.detectAndCompute(img1, None)
kp2, des2 = feature.detectAndCompute(img2, None)

# ========= BFMatcher ================
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)  # k Count of best matches found per each query descriptor
# matches = bf.Match(des1, des2)  # k Count of best matches found per each query descriptor
# matches = sorted(matches, key = lambda x:x.distance)

print(f'len(matches)\t\t= {len(matches)} 組, \tk = 2\n')

for i in range(3) :
    print(f'matches[{i}]\t\t= {matches[i]}')
    for j in range(2) :
        print(f'matches[{i}][{j}].queryIdx\t= {matches[i][j].queryIdx}\n'
              f'matches[{i}][{j}].trainIdx\t= {matches[i][j].trainIdx}\n'
              f'matches[{i}][{j}].distance\t= {matches[i][j].distance:.2f}\n')
    print('-'*80)

good = []
for m, n in matches:
    if (m.distance < .4 * n.distance):      # #如果第一個鄰近距離比第二個鄰近距離的0.4倍小，則保留 try 0.5, 0.6, 0.7
    # if m.distance / n.distance < 0.4:      # #如果第一個鄰近距離比第二個鄰近距離的0.4倍小，則保留 try 0.5, 0.6, 0.7
    # if m.distance < 100:      # #如果第一個鄰近距離
        good.append(m)
print(f'Matching points : {len(good)}')
img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, [good], outImg=None, 
        flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

img3 = cv2.resize(img3, None, fx=.8, fy=.8)
cv2.imshow('video', img3)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### orb match

In [ ]:
import cv2

# ======== example 0 =================
# img1 = cv2.imread('./image/box.png')
# img2 = cv2.imread('./image/box_in_scene.png')

# ======== example 1 =================
# img1 = cv2.imread('./image/testpic.jpg')
# img2 = cv2.imread('./image/testpic2.jpg')

# ======== example 2 =================
img1 = cv2.imread('./image/aiotbooks01.jpg')
img2 = cv2.imread('./image/aiotimage.jpg')


orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)

matches = sorted(matches, key=lambda x:x.distance)
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:60], outImg=None,
                       flags = cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

# width, height, channel = img3.shape
# ratio = float(width) / float(height)
# img3 = cv2.resize(img3, (1024, int(1024 * ratio)))
cv2.imshow('image', img3)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### FLANN 是快速最近鄰搜尋包 (Fast_Library_for_Approximate_Nearest_Neighbors) 的簡稱。
> 它是一個對大資料集和高維特徵進行最近鄰搜尋的演算法的集合,而且這些演算法都已經被優化過了。在面對大資料集時它的效果要好於 BFMatcher。

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('./image/box.png', 0)  # queryImage
img2 = cv2.imread('./image/box_in_scene.png', 0)  # trainImage

# Initiate SIFT detector
# sift = cv2.SIFT()
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# ============= FLANN parameters ===========
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)  # or pass empty dictionary

# ============= Brute-Force ============
# bf = cv2.BFMatcher()
# matches = bf.knnMatch(des1, des2, k=2)

# ============= Flann ============
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0, 0] for i in range(len(matches))]

# ratio test as per Lowe's paper
for i, (m, n) in enumerate(matches):
    if m.distance < 0.7 * n.distance:
        matchesMask[i] = [1, 0]

draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   matchesMask=matchesMask,
                   flags=0)

img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches, None, **draw_params)

plt.figure(figsize=(12, 6))
plt.imshow(img3, ), plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### homography

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

MIN_MATCH_COUNT = 10
# =======================================
# img1 = cv2.imread('./image/box.png', 0)    # Source Image
# img2 = cv2.imread('./image/box_in_scene.png', 0)    # Check Image
#=======================================
img1 = cv2.imread('./image/jp_01.png', 0)    # Source Image
img2 = cv2.imread('./image/jp_02.png', 0)    # Check Image

#=======================================
# img1 = cv2.imread('./image/douglas_01.png', 0)    # Source Image
# img2 = cv2.imread('./image/douglas_02.png', 0)    # Check Image

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1, des2, k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
   
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.flatten().tolist()

    h,w = img1.shape
    pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

else:
    print(f'Not enough matches are found - {len(good)}/{MIN_MATCH_COUNT}')
    matchesMask = None

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
        singlePointColor = None,
        matchesMask = matchesMask, # draw only inliers
        flags = 2)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

# cv2.imshow('Result', img3)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

plt.figure(figsize=(10, 8))
plt.imshow(img3, 'gray'),plt.show()

---

<div style="page-break-after: always"></div>

# Module 12. 直方圖處理

## 12-1: 直方圖的含意
> 定義 : 直方圖反應了一張圖像內所有像素強度從 `0 至 255 的分佈情形`<br>
> 特色 : 能掌握圖像的明暗、對比及像素強度分布

> Histograms 是直方圖的意思，在攝影領域一般稱為`曝光直方圖`，主要用以分析一張照片的曝光是否正確。應用在電腦視覺則主要利用它來判斷白平衡、處理 thresholding，也可用於物體追蹤（例如 CamShift 演算法即使用彩色直方圖的變化達到跟蹤目的），另外也可用於圖像檢索目的（例如 Bag-of-words 詞袋演算法）、或應用於機器學習運算…等等。因為 Histograms 為我們統計出各像素`強度頻率`的資訊來呈現整張相片的色彩分佈情況，因此我們得以透過此資訊來猜測該圖片的物件及特性。

>><img src="./image/calcHist.jpg"  style='width:60%'>

### 直方圖均衡化，用於提高影像的質量, 直方圖均衡化是通過拉伸畫素`強度分佈範圍`來增強影像對比度的一種方法.
> 所謂直方圖就是對影像的中的這些畫素點的值進行統計，得到一個統一的整體的灰度概念。直方圖的好處就在於可以清晰瞭解影像的整體灰度分佈，這對於後面依據直方圖處理影像來說至關重要。

> 在開發影像處理的程式時，我們時常會需要觀察影像像素值的分佈與特性，以便選用適合的演算法、制定門檻值、設計出適合的影像處理流程。

><img src="./image/histogram_equalization.jpg"  style='width:90%'>

<div style="page-break-after: always"></div>

## 12-2: 直方圖均衡化

> 使用 cv2.calcHist 可產生指定圖片的直方圖資訊。<br>
> cv2.calcHist ( images, channels, mask, histSize, ranges )

> |引數      |說明                                                                          |
> |---------|------------------------------------------------------------------------------|
> |images   |要分析的圖片檔，其型別可以是 uint8 或 float32，變數必須放在中`括號當中`，例如：[img]。|
> |channels |產生的直方圖類型指定影像的通道（channel)。同樣必須放在中`括號當中`例：[0]→灰階，[0, 1, 2]→RGB三色。|
> |mask     |optional，若有提供則僅計算mask的部份。，若指定為 None 則會計算整張圖形的所有像素。  |
> |histSize |要切分的像素強度值範圍，`預設為256`。每個channel皆可指定一個範圍（bins)。例如，[32,32,32] 表示RGB三個channels皆切分為32區段，也就是圖形畫出來要有幾條長方形。                                  |
> |ranges   |X軸(像素強度)的範圍，`預設為[0,256]`）。|

https://zh.wikipedia.org/zh-tw/%E7%9B%B4%E6%96%B9%E5%9B%BE%E5%9D%87%E8%A1%A1%E5%8C%96

## 12-3: 實作直方圖

### Gray image

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

o = cv2.imread('./image/lenaColor.png', 0)
cv2.imshow('original', o)
cv_hist256 = cv2.calcHist([o], [0], None, [256], [0,255])
cv_hist16 = cv2.calcHist([o], [0], None, [16], [0,255])

hist, bins = np.histogram(o.flatten(), 256, [0,256])  # 拉平
cdf = hist.cumsum()                                  # 累加 cdf : cumulate distribution function
cdf_normalized = cdf * hist.max()/ cdf.max()         # 標準化
print(hist.max(), cdf.max())

fig=plt.figure(figsize=(12, 6))
plt.subplot(221)  
plt.hist(o.flatten(), 256); plt.title('bin=256')       # flatten 將多維陣列轉換為一維陣列的功能 like flatten
plt.plot(cdf_normalized, color='r')                 # plot cdf

plt.subplot(222)  
plt.hist(o.flatten(), 16);  plt.title('bin=16')

plt.subplot(223)  
plt.plot(cv_hist256, color='y');  plt.title('cv_hist256')

plt.subplot(224)  
plt.plot(cv_hist16, color='r');  plt.title('cv_hist16')

plt.show()

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### color image : cv mean

In [ ]:
import cv2
import matplotlib.pyplot as plt

o=cv2.imread('./image/lenaColor.png')
cv2.imshow('original', o)

cv_histb = cv2.calcHist([o], [0], None, [256], [0,255])   # [0] B
cv_histg = cv2.calcHist([o], [1], None, [256], [0,255])   # [1] G
cv_histr = cv2.calcHist([o], [2], None, [256], [0,255])   # [2] R

mean, std = cv2.meanStdDev(o);    print(f'mean :\n{mean}\n\nstd :\n{std}')
fig=plt.figure(figsize=(14, 8))

plt.subplot(221)
plt.plot(cv_histb, color='b');  plt.title('cv_hist b');  plt.axvline(x=mean[0], color='r')
plt.text(mean[0]+2, cv_histb.max()*.95, f'avg.:{mean[0][0]:.2f}', color='r', fontsize=10)        

plt.subplot(222)
plt.plot(cv_histg, color='g');  plt.title('cv_hist g');  plt.axvline(x=mean[1], color='r')
plt.text(mean[1]+2, cv_histg.max()*.95, f'avg.:{mean[1][0]:.2f}', color='r', fontsize=10)        

plt.subplot(223)  
plt.plot(cv_histr, color='r');  plt.title('cv_hist r');  plt.axvline(x=mean[2], color='r')
plt.text(mean[2]+2, cv_histr.max()*.95, f'avg.:{mean[2][0]:.2f}', color='r', fontsize=10)        

color = ('b', 'g', 'r')
plt.subplot(224);  plt.title('cv_hist rgb')
for i, col in enumerate(color):
    histr = cv2.calcHist([o], [i], None, [256], [0, 256])
    plt.plot(histr, color = col)

plt.show()

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Gray Level 直方圖均衡化處理實例 Histogram Equalization (HE)
> 定義 : 通過拉伸影像的像素強度分佈範圍來`增強圖像對比度`，適用於`過曝或背光的圖片`<br>
> 缺點 : 對處理的數據不加選擇(全局處理)，如此一來`會增加背景雜訊的對比度`並且`降低有用訊號(特別亮或暗)的對比度`

經過比較可以發現均衡化後的圖像明亮的部分出現了`過曝`，失去了原本的細節。這是因為原圖整體暗的部份較多，而經過直方圖均衡化後使原本亮的地方更亮，

### gray level Histogram Equalization

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

o = cv2.imread('./image/lenaColor.png', 0)
equ=cv2.equalizeHist(o)

# cv2.imshow('original', o)
# cv2.imshow('equ', equ)
print(o.size)

hist, bins = np.histogram(o.flatten(),256,[0,256])
cdf = hist.cumsum()
cdf_normalized_o = cdf * hist.max()/ cdf.max()

hist, bins = np.histogram(equ.flatten(),256,[0,256])
cdf = hist.cumsum()
cdf_normalized_eq = cdf * hist.max()/ cdf.max()

fig=plt.figure(figsize=(14, 4))

plt.subplot(121)
plt.hist(o.flatten(), 256)
plt.plot(cdf_normalized_o, color='r')

plt.subplot(122)  
plt.hist(equ.flatten(), 256)
plt.plot(cdf_normalized_eq, color='r')
plt.show()

cv2.imshow('original', o)
cv2.imshow('gray_equ', equ)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Color 均衡化處理實例

In [ ]:
import cv2
import numpy as np

img = cv2.imread('./image/lenaColor.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

equ = cv2.equalizeHist(gray)     # 灰度圖均衡化

result1 = np.hstack((gray, equ))  # 水平拼接原圖和均衡圖
cv2.imshow('grey_equ', result1)

(b, g, r) = cv2.split(img)        # 彩色影像均衡化,需要分解通道 對每一個通道均衡化
bH = cv2.equalizeHist(b)
gH = cv2.equalizeHist(g)
rH = cv2.equalizeHist(r)

equ2 = cv2.merge((bH, gH, rH))    # 合併每一個通道

result2 = np.hstack((img, equ2)) # 水平拼接原圖和均衡圖
cv2.imshow('bgr_equ', result2)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### 自適應直方圖均衡 Adaptive Histogram Equalization AHE

> 為了提高影像的`區域性對比度`, 將影像分成`若干子塊對子塊進行HE處理`，這便是AHE（自適應直方圖均衡化）
它在每一個小區域內`（預設 8×8 ）`進行直方圖均衡化。當然，如果有噪點的話，噪點會被放大，需要對小區域內的對比度進行了限制。

> * 演算法 ：與一般的直方圖均衡 (全局) 相比，AHE 透過計算圖像每一個顯著`區域的直方圖`，重新分佈圖像的亮度值來改變影像對比度。
> * 優點 ：適合於改善影像的區域性對比度以及獲得更多的影像細節。
> * 缺點 ：在對比度增強的同時，也放大了影像的噪音，

>限制對比度自適應直方圖均衡 Contrast Limited Adaptive Histogram Equalization CLAHE 演算法 ：CLAHE與AHE都是局部均衡化，也就是把整個圖像分成許多小塊Tiles (OpenCV default為8×8)，對每個小塊進行均衡化。這種方法主要對於圖像直方圖不是那麼單一的圖像(e.g. 多峰情況)比較實用。所以在每一個的區域中，直方圖會集中在某一個小的區域中
> * clipLimit 參數表示對比度的大小。 
> * tileGridSize 參數表示每次處理塊的大小 。<br>

https://iter01.com/518513.html

In [ ]:
import cv2
import numpy as np

img = cv2.imread('./image/lenaColor.png', 0)

# 全域性直方圖均衡
equ = cv2.equalizeHist(img)

# 自適應直方圖均衡
# @param clipLimit Threshold for contrast limiting.
clahe = cv2.createCLAHE(clipLimit = 100.0, tileGridSize = (8, 8))  # AHE   # default cliplimit  = 40 
cl1 = clahe.apply(img)

# 水平拼接三張影像
result1 = np.hstack((img, equ, cl1))
cv2.imshow('clahe_result', cv2.resize(result1, (0,0), fx=.8, fy=.8))

fig=plt.figure(figsize=(12, 8))

plt.subplot(221)
plt.hist(img.flatten(), 256, [0, 255], label='original image'), plt.legend()

plt.subplot(222)
plt.hist(equ.flatten(), 256, [0, 255], label='equalize image', color='orange'), plt.legend()

plt.subplot(223)
plt.hist(cl1.flatten(), 256, [0, 255], label='clahe image', color='g'), plt.legend()

plt.subplot(224)
plt.hist(img.flatten(), 256, [0, 255], label='original image')
plt.hist(equ.flatten(), 256, [0, 255], label='equalize image')
plt.hist(cl1.flatten(), 256, [0, 255], label='clahe image')
plt.legend()

cv2.waitKey()
cv2.destroyAllWindows()

---

<div style="page-break-after: always"></div>

# Module 13. 視訊處理
> 介紹如何使用 OpenCV 擷取網路攝影機影像，處理與顯示即時的畫面影像，並將連續的畫面影像寫入影片檔案中儲存起來。網路攝影機的串流影像，可以透過 OpenCV 模組的 VideoCapture 影片擷取功能來達成，至於寫入影片檔則可使用 VideoWriter

## 13-1:視訊預覽
> 在準備擷取攝影機的影像之前，要先呼叫 cv2.VideoCapture 建立一個 VideoCapture 物件，這個 VideoCapture 物件會連接到一隻網路攝影機，我們可以靠著它的參數來指定要使用那一隻攝影機（0 代表第一隻、1 代表第二隻）。

> 建立好 VideoCapture 物件之後，就可以使用它的 read 函數來擷取一張張連續的畫面影像了。

> 在這個無窮迴圈中，每次呼叫 cap.read() 就會讀取一張畫面，其第一個傳回值 ret 代表成功與否（True 代表成功，False 代表失敗），而第二個傳回值 frame 就是攝影機的單張畫面。

>### play video file or camera preview
> * cap = VideoCapture([index, file]) 開啟相機裝置 index, 視訊檔案 file
> * retval = VideoCapture.isOpened()  : True, False 判斷視訊捕獲是否初始化成功。初始化成功返回true
> * ret, frame = cap.read() : 
> * cap.release()

In [ ]:
import numpy as np
import cv2

# cap = cv2.VideoCapture('./video/chaplin.mp4')   # play video file
cap = cv2.VideoCapture(0)                         # from camera

fps = cap.get(cv2.CAP_PROP_FPS)               # Frame Per Second
F_Count = cap.get(cv2.CAP_PROP_FRAME_COUNT)   # frame count
print(f'fps : {fps:.2f} f/s, Frame_Count : {F_Count}')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or cv2.waitKey(1) == 27: break

    frame = cv2.flip(frame, 1)   # left side right
    c=cv2.waitKey(25)            # 25 ms per frame     1/fps

#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame',frame)

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Play video in jupyter lab

In [ ]:
from IPython.display import HTML

HTML('''
    <video alt='test' controls>
        <source src='./video/chaplin.mp4' type='video/mp4'>
    </video>
''')

## 13-2: 視訊讀寫

### imWrite : image writer
* cv2.WINDOW_NORMAL : ``可以`` 讓使用者改變視窗大小
* cv2.WINDOW_AUTOSIZE : 使用者 ``不可`` 改變視窗大小
* CAP_PROP_FRAME_WIDTH, CAP_PROP_FRAME_HEIGHT : 取的影像的長寬
* cv2.VideoCapture.set( propid, value )

|類別                       | propId | 說明                                  |
|---------------------------|:------:|---------------------------------------|
|cv2.CAP_PROP_POS_MSEC      | 0      | 視訊檔案的當前位置（ms）                |
|`cv2.CAP_PROP_POS_FRAMES`    | 1      | 從0開始索引幀，幀位置。                 |
|cv2.CAP_PROP_POS_AVI_RATIO | 2      | 視訊檔案的相對位置（0表示開始，1表示結束）|
|`cv2.CAP_PROP_FRAME_WIDTH`   | 3    | 視訊流的幀寬度                         |
|`cv2.CAP_PROP_FRAME_HEIGHT`  | 4    | 視訊流的幀高度                         |
|`cv2.CAP_PROP_FPS`           | 5    | 幀率                                  |
|`cv2.CAP_PROP_FOURCC`        | 6    | 編解碼器四字元程式碼                   |
|`cv2.CAP_PROP_FRAME_COUNT`   | 7    | 視訊檔案的幀數                         |
|cv2.CAP_PROP_FORMAT        | 8      | retrieve()返回的Mat物件的格式          |
|cv2.CAP_PROP_MODE          | 9      | 後端專用的值，指示當前捕獲模式           |
|cv2.CAP_PROP_BRIGHTNESS    | 10     | 影像的亮度，僅適用於支援的相機           |
|cv2.CAP_PROP_CONTRAST      | 11     | 影像對比度，僅適用於相機                |
|cv2.CAP_PROP_SATURATION    | 12     | 影像飽和度，僅適用於相機                 |
|cv2.CAP_PROP_HUE           | 13     | 影像色調，僅適用於相機                   |
|cv2.CAP_PROP_GAIN          | 14     | 影像增益，僅適用於支援的相機             |
|cv2.CAP_PROP_EXPOSURE      | 15     | 曝光，僅適用於支援的相機                 |
|cv2.CAP_PROP_CONVERT_RGB   | 16     | 布林標誌，指示是否應將影像轉換為RGB       |

### take 1 picture manually

In [ ]:
import cv2
# import datetime as dt
from datetime import datetime as dt
now_dt = dt.now().strftime('%m/%d/%Y %H:%M:%S')

# cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
# cap = cv2.VideoCapture('./video/chaplin.mp4')   # play video file
cap = cv2.VideoCapture(0)                         # from camera

font = 2;    lt = 16
print(f'frame_w\t\t: {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}\n'
      f'frame_fps\t: {cap.get(cv2.CAP_PROP_FPS)}\n'
      f'frame_count\t: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}')

# ratio = cap.get(cv2.CAP_PROP_FRAME_WIDTH) / cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # cap.get得到相機/視訊檔案的屬性
# w = 500;  h = int(w / ratio)

# cv2.resizeWindow('frame', w, h)       # change frame size
count=0
while True:
    ret, frame = cap.read()                          #  read frame : ret: True/ False,  frame:image
    if not ret or cv2.waitKey(30) == 27: break             # wait for 1 ms     
    
    frame = cv2.flip(frame, 1)                       # 0 : 上下左右顛倒,  -1 : 上下顛倒
    
    if cv2.waitKey(1)==ord('t') or cv2.waitKey(1)==ord('T'):
        cv2.putText(frame, now_dt, (100, 300), font, 1, (0,0,255), 2, lt)
        cv2.imwrite(f'./image/frame{count}.jpg', frame)     # save frame as JPEG file          
        print(f'save image : frame{count}.jpg')
        count+=1 
        
#     frame = cv2.resize(frame, (w, h))
    cv2.putText(frame, f'{cap.get(cv2.CAP_PROP_POS_FRAMES):.0f} frames, {cap.get(cv2.CAP_PROP_POS_MSEC):.0f} ms', 
                (50, 250), font, .8, (0,0,255), 2, lt)

    cv2.imshow('frame', frame)
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)   

### capture 3 images from video

In [ ]:
import cv2
from datetime import datetime as dt
now_dt = dt.now().strftime('%m/%d/%Y %H:%M:%S')

vidcap = cv2.VideoCapture('./video/chaplin.mp4')
font = 2;    lt = 16
success, image = vidcap.read()
count = 0
while success and count < 3:
    print('Read a new frame :', success)
    success, image = vidcap.read()
    cv2.putText(image, now_dt+' cap 3', (100, 300), font, 1, (0,0,255), 2, lt)
    cv2.imwrite(f'./image/frame{count}.jpg', image)     # save frame as JPEG file      
    print(f'Write a new frame: frame{count}.jpg\n'+'='*20)
    count += 1

## 13-3: 視訊物件ROI, 追蹤和去背景

### track one obj in video file

In [ ]:
import cv2

# cap = cv2.VideoCapture('./video/vtest.avi')
# cap = cv2.VideoCapture('./video/overpass.mp4')
cap = cv2.VideoCapture('./video/car_chase_01.mp4')

tracker = cv2.TrackerCSRT_create()

# tracker = cv2.TrackerBoosting_create()   # error
# tracker = cv2.TrackerMIL_create()
# tracker = cv2.TrackerKCF_create()
# tracker = cv2.TrackerTLD_create()            # error
# tracker = cv2.TrackerMedianFlow_create()       # error
# tracker = cv2.TrackerGOTURN_create()          # error
# tracker = cv2.TrackerMOSSE_create()         # error

roi = None
while True:
    ret, frame = cap.read()
    if not ret or cv2.waitKey(40) == 27  : break
    
    if roi is None:
        roi = cv2.selectROI('frame', frame)
        if roi != (0, 0, 0, 0):
            tracker.init(frame, roi)

    success, rect = tracker.update(frame)
    if success: 
        (x, y, w, h) = [int(i) for i in rect]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('frame', frame)
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1) 

### Track one obj in live Camera

In [ ]:
import cv2, time

cap = cv2.VideoCapture(0)
tracker = cv2.TrackerCSRT_create()

# tracker = cv2.TrackerBoosting_create()   # error
# tracker = cv2.TrackerMIL_create()
# tracker = cv2.TrackerKCF_create()
# tracker = cv2.TrackerTLD_create()            # error
# tracker = cv2.TrackerMedianFlow_create()       # error
# tracker = cv2.TrackerGOTURN_create()          # error
# tracker = cv2.TrackerMOSSE_create()         # error
time.sleep(3)
roi = None
while True:
    ret, frame = cap.read()
    if not ret or cv2.waitKey(30) == 27: break
    
    frame = cv2.flip(frame, 1)
    
    if roi is None:
        roi = cv2.selectROI('frame', frame)
        if roi != (0, 0, 0, 0):
            tracker.init(frame, roi)

    success, rect = tracker.update(frame)
    if success: 
        (x, y, w, h) = [int(i) for i in rect]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('frame', frame)
cap.release()        
cv2.destroyAllWindows()
cv2.waitKey(1)

### ROI (Region Of Interest) 只處理部分有興趣的

In [ ]:
import cv2

# 指定 ROI 座標位置
RECT = ((320, 20), (570, 390))
(left, top), (right, bottom) = RECT

def roiarea(frame):                  # 取出 ROI 子畫面
    return frame[top:bottom, left:right]

def replaceroi(frame, roi):             # 將 ROI 區域貼回到原畫面
    frame[top:bottom, left:right] = roi
    return frame

cap = cv2.VideoCapture(0)               # 開啟攝影機, 讀取畫面
ratio = cap.get(cv2.CAP_PROP_FRAME_WIDTH) / cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

w = 800
h = int(w / ratio)

while True:
    ret, frame = cap.read()
    if not ret or cv2.waitKey(30) == 27: break
    frame = cv2.resize(frame, (w, h))
    frame = cv2.flip(frame, 1)

    roi = roiarea(frame)                   # 取出子畫面
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)        # BGR to HSV
#     roi = cv2.cvtColor(roi, cv2.COLOR_BGR2XYZ)          # BGR to XYZ
    
    frame = replaceroi(frame, roi)         # 將處理完的子畫面貼回到原本畫面中
    
    cv2.rectangle(frame, RECT[0], RECT[1], (0,0,255), 2)      # 在 ROI 範圍處畫個框
    cv2.imshow('frame', frame)
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1) 

---

<div style="page-break-after: always"></div>

# Module 14. OpenCV 函式庫 DLib 介紹（Python）
> Dlib 使用的人臉偵測演算法是以方向梯度直方圖 Histogram of Oriented Gradients（HOG）的特徵加上線性分類器（linear classifier）、影像金字塔（image pyramid）與滑動窗格（sliding window）來實作的

## 14-1: DLib 影像辨識應用
> dlib 是一套包含了機器學習、計算機視覺、圖像處理等的函式庫，使用 C++ 開發而成，目前廣泛使用於工業及學術界，也應用在機器人、嵌入式系統、手機、甚至於大型的運算架構中，而且最重要的是，它不但開源且完全免費，而且可跨平台使用（Linux、Mac OS、Windows），並且除了 C++ 之外還提供了 Python API ，因此如果我們想要建立一套物件偵測系統，dlib 是相當適合的平台。

## 14-2: DLib 套件應用
> * pip install dlib==19.22.99<br>
>> or (pip install C:\Users\User\Desktop\OpenCV\install\dlib-19.22.99-cp39-cp39-win_amd64.whl)

## 14-3: DLib 特徵點描述

> ### 照片人臉偵測
> detector 函數的第二個參數是指定不取樣（unsample）的次數，如果`圖片太小的時候`，將其設為 1 可讓程式偵較容易測出更多的人臉。

In [ ]:
import dlib
import cv2
print(f'dlib ver.\t: {dlib.__version__}\n\t\t: {dlib.__file__}')

In [ ]:
# img = cv2.imread('./image/dlib.jpg')   # 讀取照片圖檔
# img = cv2.imread('./image/faces02.png')
img = cv2.imread('./image/face03.jpg')

detector = dlib.get_frontal_face_detector()  # Dlib 的人臉偵測器

face_rects = detector(img, 1)   # try (img, 1)   # 偵測人臉
print(f'detected face numder : {len(face_rects)}')

# 取出所有偵測的結果
for i, d in enumerate(face_rects):
    x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA)  # 以方框標示偵測的人臉
    cv2.putText(img, f'{i}', (x1, y1-10), 0, .7, (0, 255, 0), 2, 16)
cv2.imshow('Face Detection', img)     # 顯示結果

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### 偵測結果與分數
>改用 detector.run 來偵測人臉，它的第三個參數是指定分數的門檻值，所有分數超過這個門檻值的偵測結果都會被輸出，而傳回的結果除了人臉的位置之外，還有分數（scores）與子偵測器的編號（idx），`子偵測器的編號可以用來判斷人臉的方向`，請參考 Dlib 的說明

In [ ]:
import dlib
import cv2

img = cv2.imread('./image/dlib.jpg')
# img = cv2.imread('./image/faces02.png')

detector = dlib.get_frontal_face_detector()
font = 2;    lt = 16
face_rects, scores, idx = detector.run(img, 1, -0)   # 偵測人臉，輸出分數
print(f'detected face numder : {len(face_rects)}')

for i, d in enumerate(face_rects):
    x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()

    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3, cv2.LINE_AA)
    cv2.putText(img, f'{scores[i]:.2f}({idx[i]:0.0f})', (x1, y1), font,       # 標示分數
          0.7, (0, 0, 255), 1, lt)

cv2.imshow('Face Detection', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### 影片人臉偵測
>FourCC　是一個4位元組碼，用來確定視頻的編碼格式。<br>
FOURCC is short for `FOUR Character Code` - an identifier for a video codec, compression format, color or pixel format used in media files.<br>
https://calculla.com/fourcc<br>
https://blog.csdn.net/u013943420/article/details/78779197

In [ ]:
import dlib
import cv2

cap = cv2.VideoCapture('./video/Alec_Baldwin.mp4')    # 開啟影片檔案

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))        # 取得畫面尺寸
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')             # 使用 XVID 編碼
cap.set(cv2.CAP_PROP_POS_MSEC, 10000)               #  從第 10 秒開始撥放

# 建立 VideoWriter 物件，輸出影片至 output.avi，FPS 值為 20.0
out = cv2.VideoWriter('./video/Alec_Baldwin_out.mp4', fourcc, fps, (w, h))
font = 2;    lt = 16
detector = dlib.get_frontal_face_detector()          # Dlib 的人臉偵測器

while(cap.isOpened() and cap.get(cv2.CAP_PROP_POS_MSEC)) < 20000:    # 以迴圈從影片檔案讀取影格，並顯示出來
    ret, frame = cap.read()
    if not ret or cv2.waitKey(1) & 0xFF == 27 : break
    
    face_rects, scores, idx = detector.run(frame, 0, -.5)  # 偵測人臉
    for i, d in enumerate(face_rects):               # 取出所有偵測的結果
        x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA) # 以方框標示偵測的人臉
        cv2.putText(frame, f'{scores[i]:.2f}, ({idx[i]:0.0f})', (x1, y1), font,          # 標示分數
                    0.7, (255, 255, 255), 1, lt)
            
    cv2.imshow('Face Detection', frame)           # 顯示結果
    out.write(frame)                               # 寫入影格

cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

### 即時串流影像人臉偵測

In [ ]:
import dlib
import cv2

cap = cv2.VideoCapture(0)   # 開啟影片檔案
detector = dlib.get_frontal_face_detector() # Dlib 的人臉偵測器
font = 2;    lt = 16
while(cap.isOpened()):               # 以迴圈從影片檔案讀取影格，並顯示出來
    ret, frame = cap.read()
    if not ret or cv2.waitKey(1) & 0xFF == 27 : break
    
    face_rects, scores, idx = detector.run(frame, 0)  # 偵測人臉

    for i, d in enumerate(face_rects):                  # 取出所有偵測的結果
        x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA) # 以方框標示偵測的人臉
        cv2.putText(frame, f'{scores[i]:.2f}, ({idx[i]:0.0f})', (x1, y1), font,    # 標示分數
            0.7, (255, 255, 255), 1, lt)

    cv2.imshow('Face Detection', frame)                      # 顯示結果

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

### 實現人臉 68 個關鍵點檢測
> dlib 也提供訓練好的模型，可以辨識出人臉的 68 的特徵點，68 特徵點包括鼻子、眼睛、眉毛，以及嘴巴等等，如上圖紅點就是偵測出人臉的 68 個特徵點。
$$EAR = \frac{||p_2-p_6||+||p_3-p_5||}{2||p_1-p_4||}$$

><img src="./image/dlib68.jpg"  style='width:48%'>
><img src="./image/dlib01.jpg"  style='width:48%'><br>
><img src="./image/dlib02.jpg"  style='width:48%'>
><img src="./image/dlib03.jfif"  style='width:48%'>
><img src="./image/face68_1.jpg"  style='width:60%'>
### reference :
https://www.youtube.com/watch?v=DHYM4-8x9Po

In [ ]:
import numpy as np
import cv2
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r'./model/shape_predictor_68_face_landmarks.dat')
# predictor = dlib.shape_predictor(r'./model/shape_predictor_5_face_landmarks.dat')
font = 2;    lt = 16            # 利用cv2.putText輸出1-68

img = cv2.imread('./image/face68.JFIF')     # cv2讀取影像
# img = cv2.imread('./image/lenaColor.png')     # cv2讀取影像
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # 取灰度

rects = detector(img_gray, 0)             # 人臉數rects
for i in range(len(rects)):
    landmarks = np.matrix([[p.x, p.y] for p in predictor(img,rects[i]).parts()])
    for idx, point in enumerate(landmarks):
        
        pos = (point[0, 0], point[0, 1])   # 68點的座標
#         print(idx,pos)

        cv2.circle(img, pos, 3, color=(0, 255, 0)) # 利用cv2.circle給每個特徵點畫一個圈，共68個
        cv2.putText(img, str(idx+1), pos, font, 0.8, (0, 0, 255), 1, lt)

cv2.imshow('img', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### dlib - Obj detect
> * 先收集欲辨識的人正面照片 ( 甚至可以收集多角度照片 ) 於特定資料夾 (此處設為 ./dlib_ObjectCategories10)中，並將檔名設為人名。
> * 使用 dlib.get_frontal_face_detector() 擷取資料夾中照片的人臉，再利用 dlib.shape_predictor() 取出臉部 68 個關鍵點。
> * dlib.face_recognition_model_v1().compute_face_descriptor() 將 68 個關鍵點進行嵌入成一個 128 維的向量 ν1, ν2, ⋯。
> * 相同的方式將鏡頭中的人臉也嵌入成 128 維的向量 ν。
> * 計算 ν 與 ν1, ν2, ⋯ 個別計算歐式距離，最接近者及判定其身分。

#### download : 101_ObjectCategories.tar.gz (131Mbytes)
#### dlownload : Annotations.tar
http://www.vision.caltech.edu/Image_Datasets/Caltech101/

In [ ]:
# 匯入必要的library
from scipy.io import loadmat
from skimage import io
import dlib, os, glob, re
from imutils import paths

### loading

In [ ]:
options = dlib.simple_object_detector_training_options()
images = [] # 存放相片圖檔
boxes = []  # 存放Annotations

for imagePath in paths.list_images('./dlib_ObjectCategories10'):       #依序處理path下的每張圖片

    imageID = imagePath.split('\\')[1:]     #從圖片路徑名稱中取出ImageID : according
    imageCat = imageID[1].split('.')[0].split('_')[1]  # 0001
#     print(imagePath, imageID, imageCat)

    p = f'./dlib_Annotations10/{imageID[0]}/annotation_{imageCat}.mat'   #載入Annotation
    annotations = loadmat(p)['box_coord']
    #取出annotations資訊繪成矩形物件，放入boxes變數中。
#     (x, y, w, h) = (b.left(), b.top(), b.right(), b.bottom())
    
    bb = [dlib.rectangle(left=int(x), top=int(y), right=int(w), bottom=int(h)) for (y, h, x, w) in annotations]
    boxes.append(bb)
    print(f'{imagePath},   {p}')
    images.append(io.imread(imagePath))          #將圖片放入images變數
print('done', len(images), len(boxes))

### training

In [ ]:
#丟入三個參數開始訓練

print('[INFO] training detector...')
detector = dlib.train_simple_object_detector(images, boxes, options)
 
# 將訓練結果匯出到檔案
print('[INFO] dumping classifier to file...')
detector.save('./dlib_output/model.svm')           # save

detector = dlib.simple_object_detector('./dlib_output/model.svm')   # load back

# 圖形化顯示 Histogram of Oriented Gradients（簡稱HOG）
win = dlib.image_window()
win.set_image(detector)
dlib.hit_enter_to_continue()

In [ ]:
import pickle

pickle.dump(detector, open('./dlib_output/model.pkl', 'wb'))
detector = pickle.load(open(r'./dlib_output/model.pkl', 'rb'))

### detect

In [ ]:
import dlib
import cv2
from imutils import paths
#載入訓練好的detector
# detector = dlib.simple_object_detector(detector)
 
#載入測試圖片逐張進行
for idx, testingPath in enumerate(paths.list_images('./dlib_ObjectCategories10/accordion')):
# for idx, testingPath in enumerate(paths.list_images('./dlib_ObjectCategories10/camera')):
#讀取圖片並執行dector並產生矩形物件以便用於標記辨識出的部份
    image = cv2.imread(testingPath)
    boxes = detector(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    for b in boxes:                    #在圖片上繪出該矩形
        (x, y, w, h) = (b.left(), b.top(), b.right(), b.bottom())
        cv2.rectangle(image, (x, y), (w, h), (0, 255, 0), 2)

    cv2.imshow(str(idx), image)         #顯示圖片
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

---

<div style="page-break-after: always"></div>

# Module 15. OCR 光學字元識別

## 15-1 OCR 介紹(Optical Character Recognition)
> OCR 為光學文字識別的縮寫（Optical Character Recognition，OCR），就是將`圖片翻譯為文字`。而 Tesseract 是一個 OCR 模組，目前由 Google 贊助。Tesseract 已經有 30 年歷史，一開始它是惠普實驗室的一款專利軟體，於 2005 年開源，從 2006 年後由 Google 贊助進行後續的開發和維護, Tesseract 也是目前公認最優秀、最精準的開源 OCR 系統

> Tesseract目前已作為開源項目發佈在Google Project，其最新版本3.0已經`支持中文OCR`，並提供了一個命令行工具。
主要使用在辨識掃描文件/圖片的文字。

>除了極高的精準度外，Tesseract 也有很高的靈活性，能夠通過訓練識別出任何字體（只要這些字體的風格不變就可以），也能識別出任何 Unicode 字符，是不是非常厲害呢？我們待會會用到的 pytesseract 模組就像是Tesseract的 python 包裝器。

><img src="./image/OCR1.png"  style='width:90%'>
><img src="./image/OCR2.png"  style='width:90%'>
><img src="./image/OCR.jpeg"  style='width:90%'>

## 15-2 Tesseract 安裝

> * pip install pytesseract
> * https://github.com/UB-Mannheim/tesseract/wiki <br>
>> (download : tesseract-ocr-w64-setup-v..........exe)
> * run : tesseract-ocr-w64-setup-5.3.1.20230401.exe
> * 安裝好後找到 tesseract.exe 的位置，並複製其絕對路徑，通常會在<br>
>> C:\Program Files\Tesseract-OCR\tesseract.exe。

> 安裝好後已內建英文包，到此網站下載中文的語言辨識包

>> https://github.com/tesseract-ocr/tessdata_best<br>
>> — chi_tra.traineddata → 繁體中文包<br>
>> — chi_sim.traineddata → 簡體中文包

>>下載後將檔案放到 : C:\Program Files\Tesseract-OCR\tessdata\ 中，

### run 下列指令確保 tesseract 安裝成功

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
pytesseract.get_tesseract_version()

## 15-3 實作光學字元辨識

### English

In [ ]:
import pytesseract
import cv2

# img = cv2.imread('./image/tess01.jpeg', 0)
# img = cv2.resize(img, (650, 850))

# img = cv2.imread('./image/starbucks.jpg', 0)
# img=cv2.resize(img, (850, 650))

# img = cv2.imread('./image/CarPlate1.jpg', 0)   # CarPlate1, 2
# img = cv2.imread('./image/CarPlate2.jpg', 0)   # CarPlate1, 2
img = cv2.imread('./image/CarPlate3.jpg', 0)   # CarPlate1, 2
# img=cv2.resize(img, (450, 550))

cv2.imshow('original', img)

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
text = pytesseract.image_to_string(img, lang='eng')  # chi_tra, chi_sim 修改 lang 參數變就可以
print(text)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Chi_tri : OCR 辨識繁體中文

> OCR 識別提取圖片中文字原理

> * 預處理：在這個步驟通常有：灰度化（如果是彩色影像）、降噪、二值化、字元切分以及歸一化這些子步驟。
>> * 經過二值化後，影像只剩下兩種顏色，即黑和白，其中一個是影像背景，另一個顏色就是要識別的文字了。
>> * 降噪在這個階段非常重要，降噪演算法的好壞對特徵提取的影響很大。
>> * 字元切分則是將影像中的文字分割成單個文字——識別的時候是一個字一個字識別的。如果文字行有傾斜的話往往還要進行傾斜校正。
>> * 歸一化則是將單個的文字影像規整到同樣的尺寸，在同一個規格下，才能應用統一的演算法。

> * 特徵提取和降維：對於漢字來說，特徵提取比較困難，因為首先漢字是大字符集，漢字中光是最常用的
>> * 第一級漢字就有3755個
>> * 第二漢字結構複雜，形近字多。

>> 在確定了使用何種特徵後，視情況而定，還有可能要進行特徵降維，這種情況就是如果特徵的維數太高（特徵一般用一個向量表示，維數即該向量的分量數），分類器的效率會受到很大的影響，為了提高識別速率，往往就要進行降維，這個過程也很重要，既要降低維數吧，又得使得`減少維數後的特徵向量還保留了足夠的資訊量`（以區分不同的文字）。
>分類器設計、訓練和實際識別：分類器是用來進行識別的，就是對於第二步，對一個文字影像，提取出特徵給，丟給分類器，分類器就對其進行分類，告訴你這個特徵該識別成哪個文字。

> * 後處理：後處理是用來對分類結果進行優化的，
>> * 第一個，分類器的分類有時候不一定是完全正確的（實際上也做不到完全正確），比如對漢字的識別，由於漢字中形近字的存在，很容易將一個字識別成其形近字。後處理中可以去解決這個問題，比如通過語言模型來進行校正——如果分類器將“在哪裡”識別成“存哪裡”，通過`語言模型`會發現“存哪裡”是錯誤的，然後進行校正。
>> * 第二個，OCR的識別影像往往是有大量文字的，而且這些文字存在排版、字型大小等複雜情況，後處理中可以嘗試去對識別結果進行格式化，比如按照影像中的排版排列什麼的，舉個例子，一張影像，其左半部分的文字和右半部分的文字毫無關係，而在字元切分過程中，往往是按行切分的，那麼識別結果中左半部分的第一行後面會跟著右半部分的第一行諸如此類。

In [ ]:
import pytesseract
import cv2

# img = cv2.imread('./image/OCR7.jpg', 0)
img = cv2.imread('./image/tess02.jpg', 0)

img1=cv2.resize(img, (650, 850))
cv2.imshow('original', img)

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
text = pytesseract.image_to_string(img, lang='chi_tra')  # try : chi_tra+eng, chi_sim
print(text)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
text.replace(' ', '').replace('.', ', ').replace('\n', '').replace('‧', '')

--- 

### Tesseract 命令列參數最重要的三個是 -l, --oem, --psm。
* -l ：控制輸入文本的語言，用 eng 表示英文（預設語言），用 chi_sim 表示中文簡體，用 chi_tra 表示中文繁體。
* --oem ：OCR Engine modes，Tesseract 有兩個OCR引擎，使用 -oem 選擇演算法類型，有四種操作模式可供選擇。

|值 |說明                                 |
|---|------------------------------------|
| 0 |Legacy engine only                  |
| 1 |Neural nets LSTM engine only        |
| 2 |Legacy + LSTM engines               |
| 3 |Default, based on what is available |

> 用 --oem 1 表示我們希望只使用LSTM neural network。
* --psm：Page segmentation modes，控制 Tesseract 使用的自動頁面分割模式。

>> Page segmentation modes:

|值 |說明                                                              |
|---|------------------------------------------------------------------|
| 0 |Orientation and script detection (OSD) only                       |
| 1 |Automatic page segmentation with OSD                              |
| 2 |Automatic page segmentation, but no OSD, or OCR (not implemented) |
| `3` |`Fully automatic page segmentation, but no OSD (Default)`           |
| 4 |Assume a single column of text of variable sizes                  |
| 5 |Assume a single uniform block of vertically aligned text          |
| 6 |Assume a single uniform block of text                             |
| 7 |Treat the image as a single text line                             |
| 8 |Treat the image as a single word                                  |
| 9 |Treat the image as a single word in a circle                      |
| 10| Treat the image as a single character                            |
| 11| Sparse text. Find as much text as possible in no particular order|
| 12| Sparse text with OSD                                             |
| 13| Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.|


In [ ]:
import cv2 
import pytesseract

img = cv2.imread('./image/tess01.jpeg', 0)
img=cv2.resize(img, (650, 850))

# Adding custom options
custom_config = r'--oem 3 --psm 3'
text = pytesseract.image_to_string(img, config=custom_config)
print(text)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### image_to_boxes 返回包含已識別字元及其框邊界的結果

In [ ]:
import cv2
import pytesseract

img = cv2.imread('./image/tess01.jpeg')
img=cv2.resize(img, (650, 850))

h, w, c = img.shape
boxes = pytesseract.image_to_boxes(img) 
for b in boxes.splitlines():
    b = b.split(' ')
    img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### image_to_data 定位圖片中的文字，以及該函數輸出資料的格式詳解, 包含框邊界，置信度和其他資訊的結果

In [ ]:
import cv2
import pytesseract
from pytesseract import Output

img = cv2.imread('./image/tess01.jpeg')
img=cv2.resize(img, (650, 850))
d = pytesseract.image_to_data(img, output_type=Output.DICT)
print(f'd.type : {type(d)}\n\n'
      f'd.keys() :\n{d.keys()}\n\n'
      f'd :\n{d}')

### text key in dict

In [ ]:
n_boxes = len(d['text'])
for i in range(n_boxes):
    if float(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

---

<center><h1>--- the end ---</h1></center>